In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import math

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes)
    to binary class matrix, for use with categorical_crossentropy.
    '''
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1.
    return Y


def calculate_performace(test_num, pred_y, labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] == 1:
            if labels[index] == pred_y[index]:
                tp = tp + 1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn + 1
            else:
                fp = fp + 1

    acc = float(tp + tn) / test_num
    precision = float(tp) / (tp + fp + 1e-06)
    npv = float(tn) / (tn + fn + 1e-06)
    sensitivity = float(tp) / (tp + fn + 1e-06)
    specificity = float(tn) / (tn + fp + 1e-06)
    mcc = float(tp * tn - fp * fn) / (math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) + 1e-06)
    f1 = float(tp * 2) / (tp * 2 + fp + fn + 1e-06)
    return acc, precision, npv, sensitivity, specificity, mcc, f1


In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Input,Dropout
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import scale,StandardScaler
from keras.layers import Dense,Input,Dropout
import os


#Load data
# path = "/content/drive/MyDrive/Watashara_Projects/ACE (Peptide)/"
# data_ = pd.read_csv(path + 'EN_All_clean.csv',header = None)

# data_np = np.array(data_)
# data = data_np[:, :]
# # minmax = MinMaxScaler()
# # data = minmax.fit(data_np)

# label1=np.ones((394,1))#Value can be changed
# label2=np.zeros((626,1))
# labels=np.append(label1,label2)

# from imblearn.over_sampling import SMOTE, BorderlineSMOTE
# oversample = SMOTE()
# X, y = oversample.fit_resample(data, labels)
# # y = labels


# X_train, X_ind, y_train, y_ind = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.shape)
# print(X_ind.shape)
# print(y_train.shape)
# print(y_ind.shape)
path = "/content/drive/MyDrive/Watashara_Projects/ACE/"
train_data = pd.read_csv(path + 'selected_features_allfeat/XtrainData_wSMOTE.csv')


X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

print(np.shape(X_train))

X = np.array(X_train,dtype=float)
[m1,n1] = np.shape(X)
# y = np.array(y_train)

# X =
y = y_train

fldr_path = path+'Update_results/LSTM_ACE/'
if not os.path.exists(fldr_path):
    os.makedirs(fldr_path)

sepscores = []
sepscores_ = []
ytest=np.ones((1,2))*0.5
yscore=np.ones((1,2))*0.5

model = Sequential()
model.add(LSTM(int(132), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(int(64), return_sequences=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(int(16), activation = 'relu'))
model.add(Dense(int(8), activation = 'relu'))
model.add(Dense(2, activation = 'softmax',name="Dense_2"))
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics =['accuracy'])

[sample_num,input_dim]=np.shape(X)
out_dim=2
ytest=np.ones((1,2))*0.5
yscore=np.ones((1,2))*0.5
probas_rnn=[]
tprs_rnn = []
sepscore_rnn = []
skf= StratifiedKFold(n_splits=5)
for i,(train, test) in enumerate (skf.split(X,y)):
    clf_rnn = model
    X_train_rnn=np.reshape(X[train],(-1,1,input_dim))
    X_test_rnn=np.reshape(X[test],(-1,1,input_dim))
    hist = clf_rnn.fit(X_train_rnn, to_categorical(y[train]),epochs=15)
    y_rnn_probas=clf_rnn.predict(X_test_rnn)
    probas_rnn.append(y_rnn_probas)
    y_class= categorical_probas_to_classes(y_rnn_probas)

    y_test=to_categorical(y[test])
    ytest=np.vstack((ytest,y_test))
    y_test_tmp=y[test]
    yscore=np.vstack((yscore,y_rnn_probas))

    acc, precision,npv, sensitivity, specificity, mcc,f1 = calculate_performace(len(y_class), y_class,y[test])
    mean_fpr = np.linspace(0, 1, 100)
    fpr, tpr, thresholds = roc_curve(y[test], y_rnn_probas[:, 1])
    tprs_rnn.append(np.interp(mean_fpr, fpr, tpr))
    tprs_rnn[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    sepscore_rnn.append([acc, precision,npv, sensitivity, specificity, mcc,f1,roc_auc])
    clf_rnn.save(f'{fldr_path}/' + str(i) + 'LSTM_new.keras')

row=ytest.shape[0]
ytest=ytest[np.array(range(1,row)),:]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(f'{fldr_path}/LSTM_ytest.csv')

yscore_=yscore[np.array(range(1,row)),:]
yscore_sum = pd.DataFrame(data=yscore_)
yscore_sum.to_csv(f'{fldr_path}/LSTM_yscore.csv')

scores=np.array(sepscore_rnn)
result1=np.mean(scores,axis=0)
H1=result1.tolist()
sepscore_rnn.append(H1)
result=sepscore_rnn
data_csv = pd.DataFrame(data=result)
data_csv.to_csv(f'{fldr_path}/LSTM_CV_results.csv')

(996, 325)
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6076 - loss: 0.6884
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8247 - loss: 0.6456
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9050 - loss: 0.4980
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9498 - loss: 0.3519
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9729 - loss: 0.2089
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9713 - loss: 0.1162
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9849 - loss: 0.0673
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9891 - loss: 0.0400
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9964 - loss: 0.0198
Epoch 10/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9974 - loss: 0.0111
Epoch 11/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9938 - loss: 0.0142
Epoch 12/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accur

**Indpendent test**

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.models import Sequential, load_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
import math

path = "/content/drive/MyDrive/Watashara_Projects/ACE/"
# data_ = pd.read_csv(path + 'Dts_AEDN1000.csv')
# data = data_.iloc[:, :-1].values
# labels = data_.iloc[:, -1].values
# y = labels
# shu = scale(data)
# Xt = shu
# yt = labels
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

folder_name = "Update_results/LSTM_ACE"

test_feat = pd.read_csv(path + 'selected_features_allfeat/XtestData.csv')


Xt = test_feat.iloc[:, :-1].values
yt = test_feat.iloc[:, -1].values



print(Xt.shape)
print(yt.shape)

# Xt = X_ind
# yt = y_ind
#

###########################################################################################
[sample_num, input_dim] = np.shape(Xt)
Xt = np.reshape(Xt, (-1, 1, input_dim))




sepscores = []
ytest = np.ones((1, 2)) * 0.5
yscore = np.ones((1, 2)) * 0.5

for i in range(5):

    loaded_model = load_model(path+ f'{folder_name}/' + str(i) + 'LSTM_new.keras')
    print("Loaded model from disk")

    # evaluate loaded model on test data

    y_score = loaded_model.predict(Xt)
    y_class = categorical_probas_to_classes(y_score)

    y_test = to_categorical(yt)
    ytest = np.vstack((ytest, y_test))
    y_test_tmp = yt
    yscore = np.vstack((yscore, y_score))

    acc, precision, npv, sensitivity, specificity, mcc, f1 = calculate_performace(len(y_class), y_class, y_test_tmp)
    fpr, tpr, _ = roc_curve(y_test[:, 1], y_score[:, 1])
    roc_auc = auc(fpr, tpr)
    sepscores.append([acc, sensitivity, specificity, mcc, f1, roc_auc])
    print(f'CNN:acc={acc:.6f},sensitivity={sensitivity:.6f},specificity={specificity:.6f},mcc={mcc:.6f},f1={f1:.6f},roc_auc={roc_auc:.6f}')


row=ytest.shape[0]
ytest=ytest[np.array(range(1,row)),:]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(path+ f'{folder_name}/LSTM_ytest_test.csv')

yscore_=yscore[np.array(range(1,row)),:]
yscore_sum = pd.DataFrame(data=yscore_)
yscore_sum.to_csv(path+ f'{folder_name}/LSTM_yscore_test.csv')

scores = np.array(sepscores)
result1 = np.mean(scores, axis=0)
H1 = result1.tolist()
sepscores.append(H1)
result = sepscores

data_csv = pd.DataFrame(data=result)
data_csv.to_csv(path+ f'{folder_name}/LSTM_test_results.csv')





(204, 325)
(204,)
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
CNN:acc=0.921569,sensitivity=0.881579,specificity=0.945312,mcc=0.831520,f1=0.893333,roc_auc=0.979544
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
CNN:acc=0.906863,sensitivity=0.868421,specificity=0.929687,mcc=0.800292,f1=0.874172,roc_auc=0.982833
Loaded model from disk


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
CNN:acc=0.926471,sensitivity=0.868421,specificity=0.960937,mcc=0.841781,f1=0.897959,roc_auc=0.981702
Loaded model from disk


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
CNN:acc=0.901961,sensitivity=0.855263,specificity=0.929687,mcc=0.789345,f1=0.866667,roc_auc=0.981497
Loaded model from disk
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
CNN:acc=0.916667,sensitivity=0.855263,specificity=0.953125,mcc=0.820496,f1=0.884354,roc_auc=0.981908
